In [1]:
import os
import re
from collections import Counter
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
cm = 1/2.54

# force GPU device
os.environ["CUDA_VISIBLE_DEVICES"]='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

from keras.src.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.src.layers import SimpleRNN, LSTM, Dense, Dropout, Bidirectional, Embedding, Input, RepeatVector, TimeDistributed, Reshape
from keras import Sequential, Model
from keras.losses import CategoricalCrossentropy, CategoricalFocalCrossentropy
from keras.optimizers import Adam
from keras.models import load_model
from keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy

from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import Sequence

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

from utils import *

In [2]:
data_directory = '/home/jrosendahl/datasets/cadets/sequences_export_benign_filetypes_path_ts/'
experiment_name = 'path_autoencoder'

checkpoint_path = f'saves/{experiment_name}'
log_path = f'{checkpoint_path}/log.csv'
history_path = f'{checkpoint_path}/history.npy'

# ensure directory exists
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)

In [3]:
# load data, build vocabulary

vocab = set()
X_train = []
longest_path = 0
mean_length = 0

files_loaded = 0

for filename in os.listdir(data_directory):
    with open(os.path.join(data_directory, filename), 'r') as f:
        files_loaded += 1
        if files_loaded % 50000 == 0:
            print(f"Files loaded: {files_loaded}")

        for line in f:
            line = line.split(',')
            path1 = line[4]
            path2 = line[5]
            if path1 == 'None':
                path1 = ''
            if path2 == 'None':
                path2 = ''

            # add all characters to the vocabulary
            vocab.update(path1)
            vocab.update(path2)

            longest_path = max(longest_path, len(path1))
            longest_path = max(longest_path, len(path2))

            if path1 != '':
                X_train.append(path1)
                mean_length += len(path1)
            if path2 != '':
                X_train.append(path2)
                mean_length += len(path2)

mean_length /= len(X_train)
vocab_size = len(vocab)

char_to_idx = {char: idx+1 for idx, char in enumerate(vocab)}
# add padding character
char_to_idx[''] = 0
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

print(f"Files loaded: {files_loaded}")
print(f'{len(X_train)=}')
print(f'{vocab_size=}')
print(f'{longest_path=}')
print(f'{mean_length=}')

Files loaded: 50000
Files loaded: 100000
Files loaded: 150000
Files loaded: 200000
Files loaded: 215150
len(X_train)=8161337
vocab_size=79
longest_path=167
mean_length=16.771043273914557


In [4]:
fixed_length = 50

In [5]:
def encode_paths_optimized(X_train, char_to_idx, fixed_length):
    # Initialize the array with zeros (for padding)
    encoded_array = np.zeros((len(X_train), fixed_length), dtype=int)

    # Iterate over each path and fill the appropriate positions in the array
    for i, path in enumerate(X_train):
        # Convert path to indices and fill in the array up to the fixed length
        path_indices = [char_to_idx[char] for char in path[:fixed_length]]  # Truncate to fixed_length
        assert path_indices is not None
        encoded_array[i, :len(path_indices)] = path_indices  # Place indices in the array

    return encoded_array

In [6]:
X_train_vectorized = encode_paths_optimized(X_train, char_to_idx, fixed_length)

In [7]:
print(f'{X_train_vectorized.shape=}')
print(f'{X_train_vectorized[0]=}')

X_train_vectorized.shape=(8161337, 50)
X_train_vectorized[0]=array([20, 31, 45, 73, 20, 54, 22,  5, 20, 75, 51, 11,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [110]:
latent_dim = 32
expand_dim = 64
recurrent_dim = 128


encoder_input = Input(shape=(fixed_length,))
x = Embedding(input_dim=vocab_size+1, output_dim=expand_dim)(encoder_input)
x = LSTM(recurrent_dim)(x)
encoder_output = Dense(latent_dim, activation='relu')(x)

# Create the encoder model
encoder = Model(encoder_input, encoder_output, name='encoder')

# Decoder definition
decoder_input = Input(shape=(latent_dim,))
x = Dense(expand_dim, activation='relu')(decoder_input)
x = RepeatVector(fixed_length)(x)
x = LSTM(recurrent_dim, return_sequences=True)(x)
decoder_output = TimeDistributed(Dense(vocab_size+1, activation='softmax'))(x)

# Create the decoder model
decoder = Model(decoder_input, decoder_output, name='decoder')

# Autoencoder definition
autoencoder_input = encoder_input
encoded_sequence = encoder(autoencoder_input)
decoded_sequence = decoder(encoded_sequence)

# Create the autoencoder model by combining encoder and decoder
autoencoder = Model(autoencoder_input, decoded_sequence, name='autoencoder')

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summaries
encoder.summary()
decoder.summary()
autoencoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_28 (InputLayer)     │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_16 (Embedding)        │ (None, 50, 64)         │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_32 (LSTM)                  │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 108,064 (422.12 KB)

 Trainable params: 108,064 (422.12 KB)

 Non-trainable params: 0 (0.00 B)

...

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_28 (InputLayer)     │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 32)             │       108,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 50, 80)         │       111,248 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 219,312 (856.69 KB)

 Trainable params: 219,312 (856.69 KB)

 Non-trainable params: 0 (0.00 B)

In [111]:
class SaveBestModels(Callback):
    def __init__(self, checkpoint_path, encoder, decoder, monitor='val_loss', mode='min', save_best_only=True):
        super(SaveBestModels, self).__init__()
        self.checkpoint_path = checkpoint_path
        self.encoder = encoder
        self.decoder = decoder
        self.monitor = monitor
        self.mode = mode
        self.save_best_only = save_best_only
        self.best = None

        if self.mode == 'min':
            self.best = float('inf')
        elif self.mode == 'max':
            self.best = float('-inf')

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            return

        if self.mode == 'min' and current < self.best:
            print(f"\nEpoch {epoch + 1}: {self.monitor} improved from {self.best} to {current}. Saving models.")
            self.best = current
            self.encoder.save(os.path.join(self.checkpoint_path, 'encoder.keras'))
            self.decoder.save(os.path.join(self.checkpoint_path, 'decoder.keras'))

        elif self.mode == 'max' and current > self.best:
            print(f"\nEpoch {epoch + 1}: {self.monitor} improved from {self.best} to {current}. Saving models.")
            self.best = current
            self.encoder.save(os.path.join(self.checkpoint_path, 'encoder.keras'))
            self.decoder.save(os.path.join(self.checkpoint_path, 'decoder.keras'))


In [112]:
callbacks = [
    SaveBestModels(
        checkpoint_path=checkpoint_path,
        encoder=encoder,
        decoder=decoder,
        monitor='val_loss',
        mode='min'
    ),
    EarlyStopping(
        patience=9,
        restore_best_weights=True,
        monitor='val_loss',
        verbose=1,
    ),
    ReduceLROnPlateau(
        patience=3,
        factor=0.5,
        verbose=1
    ),
    CSVLogger(log_path),
]

In [8]:
class DataGenerator(Sequence):
    def __init__(self, data, batch_size, fixed_length, vocab_size, shuffle=True):
        self.data = data
        self.batch_size = batch_size
        self.fixed_length = fixed_length
        self.vocab_size = vocab_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.data))
        self.on_epoch_end()

    def __len__(self):
        # Number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        # Generate indices for the batch
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Generate data for the batch
        X_batch = self.__data_generation(batch_indices)

        # Since it's an autoencoder, the target data is the same as the input data
        return X_batch, X_batch

    def on_epoch_end(self):
        # Shuffle indices after each epoch if shuffle is True
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch_indices):
        # Generate data for a batch of given indices
        # This is where you can customize your data loading and processing
        batch_data = [self.data[i] for i in batch_indices]

        # Convert the batch data to a numpy array (or whatever format is needed)
        X_batch = np.array(batch_data)

        # Ensure the data is padded or truncated to fixed_length
        X_batch = np.array([np.pad(x, (0, max(0, self.fixed_length - len(x))), constant_values=0)[:self.fixed_length] for x in X_batch])

        return X_batch

In [9]:
# split data 85, 10, 5
train_split = 0.85
val_split = 0.1
test_split = 0.05

train_size = int(len(X_train_vectorized) * train_split)
val_size = int(len(X_train_vectorized) * val_split)
test_size = int(len(X_train_vectorized) * test_split)

X_train_split = X_train_vectorized[:train_size]
X_val_split = X_train_vectorized[train_size:train_size+val_size]
X_test_split = X_train_vectorized[train_size+val_size:]

batch_size = 256

train_generator = DataGenerator(X_train_split, batch_size, fixed_length, vocab_size)
val_generator = DataGenerator(X_val_split, batch_size, fixed_length, vocab_size)
test_generator = DataGenerator(X_test_split, batch_size, fixed_length, vocab_size)

In [ ]:
# train autoencoder
history = autoencoder.fit(
    train_generator,
    validation_data=val_generator,
    epochs=200, 
    callbacks=callbacks  # Include the custom callback
)

# save history
with open(history_path, 'wb') as f:
    np.save(f, history.history)

In [48]:
test_paths = [
    '/home/jrosendahl/benign_filetypes_path_ts',
    '/home/jrosendahl/malicious_filetypes_path_ts',
]


# encode test_path with encoder
# def encode_paths_optimized(X_train, char_to_idx, fixed_length):

res = encode_paths_optimized(test_paths, char_to_idx, fixed_length)

In [49]:
print(res)

[[76 75 73 69 32 76 38  8 73 63 32 43 50 35 75 71 76 18 32 43 21 59 43 79
  12 21 71 32 68 58 33 32 63 79 33 35 68 75 79 68 63  0  0  0  0  0  0  0
   0  0]
 [76 75 73 69 32 76 38  8 73 63 32 43 50 35 75 71 76 69 35 71 21 16 21 73
   2 63 79 12 21 71 32 68 58 33 32 63 79 33 35 68 75 79 68 63  0  0  0  0
   0  0]]


In [50]:
transformed_paths = encoder.predict(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


In [51]:
print(transformed_paths)

[[3.3506565  0.         1.9430115  0.90554965 0.         1.2480431
  1.1075732  0.86714685 1.1801691  1.6789837  0.         0.21041426
  0.48095703 0.6075881  0.         0.        ]
 [3.2347128  0.         2.0553188  0.8587782  0.         1.1337669
  0.9993465  1.0855874  1.3916743  1.645171   0.         0.50308764
  0.2815776  0.2886235  0.         0.        ]]


In [52]:
reconstructed_paths = decoder.predict(transformed_paths)

print(reconstructed_paths)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
[[[9.86465878e-08 1.85292840e-11 2.39448905e-08 ... 5.92557503e-10
   3.42792879e-14 1.74338202e-12]
  [3.91157255e-06 2.19447682e-09 6.65792048e-01 ... 2.25916665e-06
   1.26040733e-10 1.88759168e-06]
  [3.25647271e-14 1.20814097e-14 2.76604289e-04 ... 2.91658191e-12
   4.51178960e-15 2.17608727e-14]
  ...
  [9.99788821e-01 4.95611253e-17 1.35953965e-12 ... 1.58455538e-13
   5.58775497e-08 1.12880295e-14]
  [9.99992847e-01 2.20202634e-18 1.16357075e-15 ... 3.19679255e-15
   8.51536008e-10 8.27269869e-16]
  [9.99994516e-01 5.00004831e-19 1.65936920e-15 ... 4.22274247e-16
   1.43458767e-09 8.64706978e-16]]

 [[1.49066253e-07 2.97048497e-11 5.29830935e-08 ... 7.54988072e-10
   6.00212629e-14 1.41899036e-12]
  [1.97131999e-06 1.26027910e-09 8.31270576e-01 ... 1.23633049e-06
   1.71489239e-10 2.78861364e-07]
  [1.69728272e-13 5.58977913e-14 5.67151094e-03 ... 2.32912162e-11
   8.02207693e-14 4.02713670e-13]
  ...
  [9.98809934e-01 7.02600489e-16 6.514

In [ ]:
# transform back to strings
reconstructed_paths = np.argmax(reconstructed_paths, axis=2)

In [55]:
reconstructed_paths = [''.join([idx_to_char[idx] for idx in path]) for path in reconstructed_paths]

In [56]:
print(reconstructed_paths)

['/uom//uprre/aassssssssssssseettttttttttttt', '/uoc/parae/aaaasssssssseeeeetttttttttttttttt']


In [18]:
# load encoder, decoder models
encoder = load_model(f'{checkpoint_path}/encoder.keras')
decoder = load_model(f'{checkpoint_path}/decoder.keras')

# build autoencoder model
autoencoder_input = encoder.input
encoded_sequence = encoder(autoencoder_input)
decoded_sequence = decoder(encoded_sequence)
autoencoder = Model(autoencoder_input, decoded_sequence, name='autoencoder')
autoencoder.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
def encode_path(path: str, char_to_idx: dict, fixed_length: int, encoder: Model):
    "take a path as a string, return the result from the encoder"
    path_indices = [char_to_idx[char] for char in path[:fixed_length]]
    path_indices = np.array(path_indices)
    path_indices = np.pad(path_indices, (0, max(0, fixed_length - len(path_indices))), constant_values=0)[:fixed_length]
    path_indices = np.array([path_indices])
    return encoder.predict(path_indices)

def decode_path(encoded_path: np.ndarray, decoder: Model, idx_to_char: dict):
    "take the result from the encoder, return the decoded path"
    decoded_path = decoder.predict(encoded_path)
    decoded_path = np.argmax(decoded_path, axis=2)
    decoded_path = [''.join([idx_to_char[idx] for idx in path]) for path in decoded_path]
    return decoded_path

In [14]:
test = encode_path('/dev/null', char_to_idx, fixed_length, encoder)
test

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


array([[0.19452104, 0.269752  , 0.        , 0.        , 0.        ,
        0.        , 2.180835  , 0.        , 0.        , 0.        ,
        0.        , 0.01902695, 0.        , 0.        , 0.        ,
        0.24062942, 2.0384183 , 0.06973484, 1.1193109 , 0.        ,
        1.5873064 , 0.83516556, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.0387359 ,
        0.        , 1.2860181 ]], dtype=float32)

In [15]:
test = decode_path(test, decoder, idx_to_char)
test

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step


['h5/-hh<<FxQ']

In [20]:
# evaluate autoencoder on X_test
test_generator = DataGenerator(X_test_split, batch_size, fixed_length, vocab_size, shuffle=False)

# Evaluate the model on the test data
test_result = autoencoder.evaluate(test_generator)

print(f'Test logg: {test_result[0]}\nTest accuracy: {test_result[1]}')

  12/1594 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - accuracy: 0.7017 - loss: 4.4755

/home/jrosendahl/.virtualenvs/models/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1594/1594 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.6386 - loss: 5.1317
Test result: [6.136971950531006, 0.5515469312667847]
